# R-SHAP Implementation

### Workspace Prep

In [1]:
import lstm_model as lm
import torch
import torch.nn as nn
import torch.nn.functional as fn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger as TensorBoardLogger
import tensorboard


import numpy as np


c:\Users\patbo\.conda\envs\theoryML\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Uncomment below to install missing modules

In [2]:
# pip install tensorboard

In [3]:
# pip install --ignore-installed PyYAML  # might be necessary to run before installing pytorch

In [4]:
# pip install pytorch-lightning

In [5]:
# pip install torch

### Load Pre-processing Utilities

In [3]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, r'D:\Documents\GitHub\itcs-8156\utils')
# sys.path.insert(1, r'H:\My Drive\SP23\ML\envs\groupWork\utils')


from preprocessing import (market_prepro,
                           lstm_timeseries_feat_and_targ,
                           
)

### Load and Pre-processs Stock Data

In [4]:
# st = "Stocks"
st = "ETFs"

#Input stock name
sn = "aadr" 
f = r'D:\Desktop\College Spring 2023\machineLearning\project\coding\data'
# f = r'H:\My Drive\stockMarket_data'
X_train, X_test, T_train, T_test = market_prepro(f,st,sn,False,splitdata=True, stdzr='minmax')

print(X_train.shape)
print(T_train.shape)
# X,T = market_prepro(f,st,sn,False,splitdata=False)

#number of days as features
day_feat = 2

#number of days to use as features
day_targ = 1
day_targ = day_targ - 1

# dl_train, ds_train = lstm_timeseries_feat_and_targ(X_train[['Open','Low']], T_train, 4, 1,None)
# dl_test, ds_test = lstm_timeseries_feat_and_targ(X_test[['Open','Low']], T_test, 4, 1,  None)

dl_train, ds_train = lstm_timeseries_feat_and_targ(X_train, T_train, day_feat, day_targ, [ 'Year', 'Month' ,'Day_date', 'Day'])
dl_test, ds_test = lstm_timeseries_feat_and_targ(X_test, T_test, day_feat, day_targ, [ 'Year', 'Month' ,'Day_date', 'Day'])


(1044, 11)
(1044,)


### Use LSTM on Stock Data

In [5]:
print(torch.version.cuda)

11.6


In [6]:
mdl_stock = lm.BasicLSTM(num_feat=7, num_hiddens=1, num_out=1, lr=0.01)

In [8]:
ds_train[0][0]

tensor([[ 0.0114,  0.0204],
        [ 0.0111,  0.0201],
        [ 0.0023,  0.0146],
        [ 0.4081,  0.1699],
        [ 0.0088,  0.0054],
        [-0.0114, -0.0047],
        [ 0.0000,  0.0000]], dtype=torch.float64)

In [7]:
mdl_stock.forward(ds_train[0][0])[0]

ValueError: setting an array element with a sequence.

In [21]:
logger = TensorBoardLogger("lightning_logs", name="market")

trainer = pl.Trainer(max_epochs=10,logger=logger) # with default learning rate, 0.001 (this tiny learning rate makes learning slow)
trainer.fit(mdl_stock, train_dataloaders=dl_train)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type | Params
------------------------------
------------------------------
38        Trainable params
0         Non-trainable params
38        Total params
0.000     Total estimated model params size (MB)


Epoch 0:   0%|                                                                                | 0/1043 [00:00<?, ?it/s]

TypeError: unsupported operand type(s) for -: 'tuple' and 'Tensor'

In [ ]:
trainer.test(mdl_stock,dataloaders=dl_test)

In [ ]:
def makepred(model, dataset):
    y = []
    t = []
    for ii in dataset:
        feat, lab = ii

        y.append(model.forward(feat).detach().numpy()[0])
        t.append(lab.numpy()[0])


    return y, t


In [ ]:
y_test, t_test = makepred(mdl_stock, ds_test)
y_train, t_train = makepred(mdl_stock, ds_train)

In [ ]:
from sklearn.metrics import (mean_absolute_percentage_error,
                             r2_score,
)

# train_mape = mean_absolute_percentage_error(t_train, y_train)
# test_mape = mean_absolute_percentage_error(t_test, y_test)



# print('Training MAPE ', train_mape)
# print('Testing MAPE ', test_mape)

print('Training r2 ', r2_score(t_train, y_train) )
print('Testing r2 ', r2_score(t_test, y_test) )

In [ ]:
import matplotlib.pyplot as plt

def scatter_results(Y,T,title):
    plt.figure(figsize=(9,9))
    plt.scatter(Y,T)
    plt.xlabel('Model Prediction')
    plt.ylabel('True Value')
    plt.title(title)

In [ ]:
scatter_results(y_test, t_test, 'Testing')
scatter_results(y_train, t_train, 'Training')

In [ ]:
plt.plot(range(len(t_test)),t_test)
plt.plot(range(len(y_test)),y_test)